In [1]:
%env CUDA_VISIBLE_DEVICES 0

import json
import os
import random

from datasets import load_from_disk
import evaluate
import numpy as np
import torch
from transformers import VisionEncoderDecoderModel, AutoImageProcessor, AutoTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments, EarlyStoppingCallback

device = "cuda"
torch.backends.cuda.matmul.allow_tf32 = True

env: CUDA_VISIBLE_DEVICES=0


In [2]:
encoder = "facebook/timesformer-base-finetuned-k600"
decoder = "gpt2"

image_processor = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base")
tokenizer = AutoTokenizer.from_pretrained(decoder)
tokenizer.pad_token = tokenizer.eos_token

model = VisionEncoderDecoderModel.from_encoder_decoder_pretrained(encoder, decoder).to(device)
model.config.decoder_start_token_id = tokenizer.bos_token_id
model.config.pad_token_id = tokenizer.pad_token_id
model.config.max_length = 50
model.config.num_beams = 4
model.config.early_stopping = True

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
Some weights of the model checkpoint at facebook/timesformer-base-finetuned-k600 were not used when initializing TimesformerModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing TimesformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TimesformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.9.crossattention.bias', 'h.4.crossattention.q_attn.weight', 'h.7.crossa

In [3]:
dataset = load_from_disk("/data1/caelen/dataset/k600_16frames_10captions")
dataset.set_format("torch")

dataset["train"] = dataset["train"].select(np.arange(6))
dataset["validation"] = dataset["validation"].select(np.arange(6))

dataset

DatasetDict({
    train: Dataset({
        features: ['video_id', 'pixel_values', 'labels'],
        num_rows: 6
    })
    validation: Dataset({
        features: ['video_id', 'pixel_values', 'labels'],
        num_rows: 6
    })
})

In [4]:
output_dir="training/test"

training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    tf32=True,
    predict_with_generate=True,
    load_best_model_at_end=True,
    logging_strategy="epoch",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    remove_unused_columns=False,
    per_device_train_batch_size=6,
    per_device_eval_batch_size=6,
    dataloader_num_workers=8,
    num_train_epochs=1,
    learning_rate=5e-7,
)

def collator(examples):
    print(examples[0].keys())
    return None
    
    pixel_values, labels = [], []
    for example in examples:
        print(example["video_id"])
        
        # train
        if len(example["pixel_values"]) == 16:
            frame_idxs = []
            for i in range(0, 16, 2):
                frame_idxs.append(i + random.randint(0, 1))
            pixel_values.append(example["pixel_values"][frame_idxs])
            labels.append(example["labels"][0])
        # val
        else:
            pixel_values.append(example["pixel_values"])
            labels.append(example["labels"])

    pixel_values = torch.stack(pixel_values)
    labels = torch.stack(labels)
    return {"pixel_values": pixel_values, "labels": labels}

bleu = evaluate.load("bleu")
meteor = evaluate.load("meteor")
rouge = evaluate.load("rouge")
val_output = {}
with open("dataset/longestCaption_videoID.json") as file:
    longestCaption_videoID = json.load(file)
with open("dataset/videoID_captions.json") as file:
    videoID_captions = json.load(file)
    
def metrics(eval_predictions):
    predictions, labels = eval_predictions
    predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    references = []
    for i, label in enumerate(labels):
        video_id = longestCaption_videoID[label]    
        references.append(videoID_captions[video_id])
        if video_id in val_output:
            val_output[video_id].append(predictions[i])
        else:
            val_output[video_id] = [predictions[i]]
            
    with open(os.path.join(output_dir, "val_output.json"), "w") as file:
        json.dump(val_output, file)
            
    bleu_scores = bleu.compute(predictions=predictions, references=references, smooth=True)
    meteor_scores = meteor.compute(predictions=predictions, references=references)
    rouge_scores = rouge.compute(predictions=predictions, references=references, rouge_types=['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)    
    return {"bleu": bleu_scores["bleu"], "meteor": meteor_scores["meteor"], "rouge1": rouge_scores["rouge1"], "rouge2": rouge_scores["rouge2"], "rougeL": rouge_scores["rougeL"]}

trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    data_collator=collator,
    compute_metrics=metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=5)],
)

trainer.train()

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/922201615/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/922201615/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/922201615/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
/home/922201615/anaconda3/envs/vlm/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 6
  Num Epochs = 1
  Instantaneous batch size per device = 6
  Total train batch size (w. parallel, distributed & accumulation) = 6
  Gradient Accumulation steps = 1
  Total optimization steps = 1
  Number of trainable

dict_keys(['video_id', 'pixel_values', 'labels'])


TypeError: object of type 'NoneType' has no len()